In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import PyMieScatt as ps
import subprocess
import math
import sys
import pandas as pd
import xgboost as xgb

import tensorflow as tf
from tensorflow import keras
from scipy import interpolate

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
lw, ms = 1, 0.02

#"np.save(data.npy)"

In [ ]:
diam_data = np.array(pd.read_csv("Datasheet/diam_entree.csv"))

indreal = [1.33, 1.385, 1.44, 1.495, 1.55]
indimg = [0.001, 0.00075025, 0.0005005, 0.00025075, 1e-06]
sig = [1.6, 1.7, 1.8, 1.9, 2.0]
Data = np.zeros(shape=(5, 5, 5, 99, 3))

for idx_i, i in enumerate(indreal):
    for idx_j, j in enumerate(indimg):
        for idx_k, k in enumerate(sig):
            filename = f"Datasheet/BAE_sortie_real_{i}_im_{j}_sig_{k}.csv"
            BAE_data = np.array(pd.read_csv(filename, delimiter=","))
            Data[idx_i, idx_j, idx_k] = BAE_data
            

In [ ]:
for i in range(5):
    for j in range(5):
        for k in range(5):
            plt.semilogx(diam_data, Data[i, j, k])
            plt.title(f"indreal={indreal[i]}, indimg={indimg[j]}, sig={sig[k]}")
            plt.xlabel("Diameter")
            plt.ylabel("BAE")
            plt.show()

In [ ]:
for idx_i, i in enumerate(indreal):
    for idx_j, j in enumerate(indimg):
        for idx_k, k in enumerate(sig):
            filename = f"Datasheet/BAE_sortie_real_{i}_im_{j}_sig_{k}.csv"
            BAE_data = np.array(pd.read_csv(filename, delimiter=","))
            Data[idx_i, idx_j, idx_k] = BAE_data
            
            input_d = np.column_stack((diam_data, np.full((diam_data.shape[0], 1), k),
                                          np.full((diam_data.shape[0], 1), i),
                                          np.full((diam_data.shape[0], 1), j)))

            output_d = BAE_data

            # Séparation des données en ensemble d'entraînement et de validation
            train_input, val_input, train_output, val_output = train_test_split(input_d, output_d, test_size=0.2, random_state=42) 

            # Création du modèle random forest
            model = RandomForestRegressor(n_estimators=100)

            # Entraînement du modèle
            model.fit(train_input, train_output)

            # Génération de nouvelles données pour tracer les prédictions
            diametre_nouveau = np.linspace(np.min(diam_data), np.max(diam_data), 100000)
            input_nouveau = np.column_stack((diametre_nouveau, np.full((diametre_nouveau.shape[0], 1), k),
                                          np.full((diametre_nouveau.shape[0], 1), i),
                                          np.full((diametre_nouveau.shape[0], 1), j)))
            BAE_nouveau = model.predict(input_nouveau)

            # Tracé des prédictions
            plt.semilogx(diam_data, BAE_data, label= ["BAE_1064_532","BAE_532_355","BAE_1064_355"])
            plt.semilogx(diametre_nouveau, BAE_nouveau, 'o-', lw = lw, ms = ms, label=["pred_BAE_1064_532","pred_BAE_532_355","pred_BAE_1064_355"])
            plt.xlabel("Diamètre d'entrée")
            plt.ylabel("BAE Random Forest")
            plt.legend()
            plt.show()

In [ ]:
for idx_i, i in enumerate(indreal):
    for idx_j, j in enumerate(indimg):
        for idx_k, k in enumerate(sig):
            filename = f"Datasheet/BAE_sortie_real_{i}_im_{j}_sig_{k}.csv"
            BAE_data = np.array(pd.read_csv(filename, delimiter=","))
            Data[idx_i, idx_j, idx_k] = BAE_data
            
            diam_data = diam_data.reshape(-1, 1)
            BAE_sortie = np.array(BAE_data)

            # Instancier et entrainer le modèle k-NN
            knn = KNeighborsRegressor(n_neighbors=10)
            knn.fit(diam_data, BAE_sortie)

            # Prédire les valeurs de BAE en utilisant le modèle k-NN
            diam_pred = np.linspace(diam_data.min(), diam_data.max(), 100000).reshape(-1, 1)
            BAE_pred = knn.predict(diam_pred)

            # Afficher les courbes de prédiction
            plt.semilogx(diam_data, BAE_sortie, label= ["BAE_1064_532","BAE_532_355","BAE_1064_355"])
            plt.semilogx(diam_pred, BAE_pred, 'o-', lw = lw, ms = ms, label=["pred_BAE_1064_532","pred_BAE_532_355","pred_BAE_1064_355"])
            plt.xscale('log')
            plt.xlabel('Diamètre')
            plt.ylabel('BAE k-NN')
            plt.legend()
            plt.show()


In [ ]:
for idx_i, i in enumerate(indreal):
    for idx_j, j in enumerate(indimg):
        for idx_k, k in enumerate(sig):
            filename = f"Datasheet/BAE_sortie_real_{i}_im_{j}_sig_{k}.csv"
            BAE_data = np.array(pd.read_csv(filename, delimiter=","))
            Data[idx_i, idx_j, idx_k] = BAE_data
            # Séparation des données en entrée et sortie
            diam_data = diam_data.reshape(-1, 1)
            BAE_sortie = np.array(BAE_data)

            # Séparation des données en ensemble d'entraînement et de validation
            train_input, val_input, train_output, val_output = train_test_split(diam_data, BAE_data, test_size=0.2, random_state=42) 

            # Conversion des données en format Dmatrix pour XGBoost
            dtrain = xgb.DMatrix(train_input, label=train_output)
            dval = xgb.DMatrix(val_input, label=val_output)

            # Paramétrage de XGBoost
            params = {'objective': 'reg:linear',
                      'eval_metric': 'rmse',
                      'seed': 42}

            # Entraînement du modèle
            model = xgb.train(params, dtrain, num_boost_round=100)

            # Évaluation du modèle
            train_error = model.eval(dtrain)[1]
            val_error = model.eval(dval)[1]

            # Génération de nouvelles données pour tracer les prédictions
            diametre_nouveau = np.linspace(np.min(diam_data), np.max(diam_data), 10000).reshape(-1, 1)
            dtest = xgb.DMatrix(diametre_nouveau)
            BAE_nouveau = model.predict(dtest)

            # Tracé des prédictions
            plt.semilogx(diam_data, BAE_data, label= ["BAE_1064_532","BAE_532_355","BAE_1064_355"])
            plt.semilogx(diametre_nouveau, BAE_nouveau, 'o-', lw = lw, ms = ms, label=["pred_BAE_1064_532","pred_BAE_532_355","pred_BAE_1064_355"])
            plt.xlabel("Diamètre d'entrée")
            plt.ylabel("BAE XGBoost")
            plt.legend()
            plt.show()

In [ ]:
for idx_i, i in enumerate(indreal):
    for idx_j, j in enumerate(indimg):
        for idx_k, k in enumerate(sig):
            filename = f"Datasheet/BAE_sortie_real_{i}_im_{j}_sig_{k}.csv"
            BAE_data = np.array(pd.read_csv(filename, delimiter=","))
            Data[idx_i, idx_j, idx_k] = BAE_data

            #Ajuster des splines sur les points de données
            BAE_splines = [interpolate.interp1d(diam_data.squeeze(), BAE_data[:, idx], kind='cubic') 
                           for idx in range(3)]
            
            #Estimer les BAE pour des diamètres qui ne sont pas dans la liste originale
            BAEs_new = np.concatenate([BAE_spline(diam_data).reshape(-1,1) for BAE_spline in BAE_splines], axis=1)

            plt.semilogx(diam_data, BAE_data, label= ["BAE_1064_532","BAE_532_355","BAE_1064_355"])
            plt.semilogx(diam_data, BAEs_new, 'o-', lw = lw, ms = ms, label=["pred_BAE_1064_532","pred_BAE_532_355","pred_BAE_1064_355"])
            plt.xscale('log')
            plt.xlabel('Diamètre')
            plt.ylabel("Approximation par Splines D'interpolation")
            plt.legend()
            plt.show()